<a href="https://colab.research.google.com/github/bob8dod/DeepLearning-with-PyTorch/blob/main/Assignments/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 실습을 위해 파이토치의 도구들을 임포트하는 기본 셋팅을 진행
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1) # 랜덤시드 고정

In [ ]:
# 실습에 사용될 학습 데이터 (다중 선형 회귀용 데이터)
X = torch.FloatTensor([[-10, 0, -8, -8],
    [-4, -3,  9,  3],
    [-7,  9, 10,  4],
    [-6, -4,  2,  3],
    [-6, -9, -9,  1],
    [-2,  5, -7, -2],
    [-9, -2, -3, -3],
    [9,  -4,  6,  4],
    [9,  -1, -9, -6],
    [1, -9, -11,  3]])
Y = torch.FloatTensor([[-2.0], [-53.9], [-21.1], [-36.7], [-15.2], [42.9], [-25.7], [-11.4], [56.0], [7.2]])

# 직접 구현한 버전

선형 회귀를 좀 더 직접적으로 이해하기 위해 가설, 비용 함수를 직접 정의해서 선형 회귀 모델을 구현

In [ ]:
print(X.shape) #torch.Size([10, 4])
print(Y.shape) #torch.Size([10, 1])

torch.Size([10, 4])
torch.Size([10, 1])


In [ ]:
'''
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
'''
# 가중치 w와 편향 b 초기화
W = torch.zeros((4, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [ ]:
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

In [ ]:
epochs = 20000
for epoch in range(1,epochs + 1):

    # H(x) 계산
    # 편향 b는 브로드 캐스팅되어 각 샘플에 더해집니다.
    hypothesis = X.matmul(W) + b

    # cost 계산
    cost = torch.mean((hypothesis - Y) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 2000 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, epochs, cost.item()))

Epoch 2000/20000 Cost: 104.073524
Epoch 4000/20000 Cost: 37.490685
Epoch 6000/20000 Cost: 23.329142
Epoch 8000/20000 Cost: 16.694483
Epoch 10000/20000 Cost: 12.622404
Epoch 12000/20000 Cost: 9.990808
Epoch 14000/20000 Cost: 8.272963
Epoch 16000/20000 Cost: 7.146646
Epoch 18000/20000 Cost: 6.404676
Epoch 20000/20000 Cost: 5.912744


In [ ]:
print(Y.squeeze())
print(list(map(lambda x: round(float(x),1), hypothesis.squeeze().detach())))

tensor([ -2.0000, -53.9000, -21.1000, -36.7000, -15.2000,  42.9000, -25.7000,
        -11.4000,  56.0000,   7.2000])
[-0.0, -55.2, -22.0, -39.3, -18.5, 38.5, -22.6, -10.8, 54.4, 9.0]


# nn.Module 및 class로 구현한 버전

파이토치에서 이미 구현되어져 제공되고 있는 함수들을 불러오는 것으로 더 쉽게 선형 회귀 모델을 구현

In [ ]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 1) # 다중 선형 회귀이므로 input_dim=4, output_dim=1.

    def forward(self, x):
        return self.linear(x)

In [ ]:
model = MultivariateLinearRegressionModel()

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5) 

In [ ]:
nb_epochs = 20000
for epoch in range(1,nb_epochs+1):

    # H(x) 계산
    prediction = model(X)
    # model(x_train)은 model.forward(x_train)와 동일함.

    # cost 계산
    cost = F.mse_loss(prediction, Y) # <== 파이토치에서 제공하는 평균 제곱 오차 함수

    # cost로 H(x) 개선하는 부분
    # gradient를 0으로 초기화
    optimizer.zero_grad()
    # 비용 함수를 미분하여 gradient 계산
    cost.backward()
    # W와 b를 업데이트
    optimizer.step()

    if epoch % 2000 == 0:
    # 100번마다 로그 출력
      print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch 2000/20000 Cost: 105.820030
Epoch 4000/20000 Cost: 36.897923
Epoch 6000/20000 Cost: 22.930859
Epoch 8000/20000 Cost: 16.648106
Epoch 10000/20000 Cost: 12.815602
Epoch 12000/20000 Cost: 10.329041
Epoch 14000/20000 Cost: 8.693331
Epoch 16000/20000 Cost: 7.608859
Epoch 18000/20000 Cost: 6.883208
Epoch 20000/20000 Cost: 6.391644


In [ ]:
print(list(model.parameters()))

[Parameter containing:
tensor([[ 2.8455,  3.7707, -3.6394,  0.0773]], requires_grad=True), Parameter containing:
tensor([-0.2828], requires_grad=True)]


In [ ]:
new_var =  torch.FloatTensor([[1, -9, -11,  3]]) 
# 입력한 값 [1, -9, -11,  3]에 대해서 예측값 y를 리턴받아서 pred_y에 저장
pred_y = model(new_var) 
print("훈련 후 입력이 1, -9, -11,  3일 때의 예측값 :", pred_y) 
print('정답:', Y[-1])

훈련 후 입력이 1, -9, -11,  3일 때의 예측값 : tensor([[8.8918]], grad_fn=<AddmmBackward>)
정답: tensor([7.2000])
